In [19]:
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
user_input = "get flight details from las vegas to delhi for 13-08-2024"

system_input = """
The user will provide flight search details in natural language. Your task is to:
extract the city names in IATA codes and Date in (YYYY-MM-DD) format 
For example : 
input : "get flight details from new york to los angeles for today"
output: search_flights("NYC","LAX","2024-08-03")
strict to the output format 
"""

completion = client.chat.completions.create(
  model="RichardErkhov/meta-llama_-_Meta-Llama-3-8B-Instruct-gguf",
  messages=[
    {"role": "system", "content": system_input},
    {"role": "user", "content": user_input}
  ],
  temperature=0.7, 
  max_tokens=18
)

# Extract the output content
response = completion.choices[0].message.content.strip()

# Remove leading and trailing backticks and any additional newlines
clean_response = response.strip("` \n")

# Print the cleaned response
print(clean_response)


search_flights("LAS", "DEL", "2024-08-13")


In [20]:
import json
import tempfile
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor
from amadeus import Client, ResponseError

# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

AMADEUS_API_KEY = 'Q9GWN9iAAmRnIPhEZFDw5qWAkyhyKxEF'
AMADEUS_API_SECRET = 'SgPgcnlNT9AXQiKF'

# Initialize the Amadeus API client
amadeus = Client(
    client_id=AMADEUS_API_KEY,
    client_secret=AMADEUS_API_SECRET
)

def search_flights(origin, destination, departure_date):
    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=origin,
            destinationLocationCode=destination,
            departureDate=departure_date,
            adults=1
        ).data
        return response
    except ResponseError as error:
        print(f"Error: {error}")
        return None

# Create a local command line code executor.
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

origin = "LAS"
destination = "DEL"
departure_date = "2024-08-13"

# Call the search_flights function with formatted parameters
clean_response = search_flights(origin, destination, departure_date)

# Print the result in JSON format
if clean_response:
    formatted_json = json.dumps(clean_response, indent=4)
    print(formatted_json)
else:
    print("No flights found or an error occurred.")

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="NEVER",  # Always take human input for this agent for safety.
)

message_with_code_block = f"""This is a message with code block.
The code block is below:
```python
import json

# Replace `clean_response` with the actual variable containing flight data
flights = {clean_response}
if flights:
    print(json.dumps(flights, indent=4))
else:
    print("No flights found or an error occurred.")
This is the end of the message
"""
reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)



[
    {
        "type": "flight-offer",
        "id": "1",
        "source": "GDS",
        "instantTicketingRequired": false,
        "nonHomogeneous": false,
        "oneWay": false,
        "isUpsellOffer": false,
        "lastTicketingDate": "2024-08-13",
        "lastTicketingDateTime": "2024-08-13",
        "numberOfBookableSeats": 7,
        "itineraries": [
            {
                "duration": "PT22H7M",
                "segments": [
                    {
                        "departure": {
                            "iataCode": "LAS",
                            "terminal": "3",
                            "at": "2024-08-13T15:38:00"
                        },
                        "arrival": {
                            "iataCode": "SFO",
                            "terminal": "2",
                            "at": "2024-08-13T17:15:00"
                        },
                        "carrierCode": "AS",
                        "number": "133",
               

In [17]:
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
user_input = json.dumps(reply)


system_input = """
You will receive JSON data containing information about flight details. Your task is to extract and convert the key details into a human-readable format.

1. Extract the departure and arrival cities with their respective IATA codes.
2. Extract the departure and arrival dates and times.
3. Extract the total price and currency.
4. Format these details into a clear and concise human-readable format.

Ensure the output is easy for users to understand and provides all the necessary details.
"""

completion = client.chat.completions.create(
  model="RichardErkhov/meta-llama_-_Meta-Llama-3-8B-Instruct-gguf",
  messages=[
    {"role": "system", "content": system_input},
    {"role": "user", "content": user_input}
  ],
  temperature=0.7,
  max_tokens=300
)

# Extract the output content
response = completion.choices[0].message.content.strip()

# Print the cleaned response
print(response)


{ "flight": [ { ... } ] }
### Response:
{ "flight": [ { ... } ] }

### Flight Details:

* departure date: '2023-01-29'
* arrival date: '2023-02-03'
* total price: 450
* currency: USD
* departure city: New York, IATA code: JFK
* arrival city: Los Angeles, IATA code: LAX

### Flight Details:

* departure date: '2023-01-29'
* arrival date: '2023-02-03'
* total price: 450
* currency: USD
* departure city: New York, IATA code: JFK
* arrival city: Los Angeles, IATA code: LAX

### Flight Details:

* departure date: '2023-01-29'
* arrival date: '2023-02-03'
* total price: 450
* currency: USD
* departure city: New York, IATA code: JFK
* arrival city: Los Angeles, IATA code: LAX

### Flight Details:

* departure date: '2023-01-29'
* arrival date: '2023-02-03'
* total price: 450
* currency: USD
* departure city: New York, IATA code: JFK
* arrival city: Los Angeles, IATA code: LAX

### Flight Details:

* departure date:
